In [21]:
from dotenv import load_dotenv
load_dotenv()
import os
import yaml
from crewai import Agent, Task, Crew,LLM
ollama_llm = LLM(
            model="ollama/llama3.1:8b",#"ollama/llama3.2:latest",
            #model="ollama/gemma3:4b",
            api_base="http://localhost:11434"
            )

### read configs

In [22]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

In [23]:
from typing import List
from pydantic import BaseModel, Field

class TaskEstimate(BaseModel):
    task_name: str = Field(..., description="Name of the task")
    estimated_time_hours: float = Field(..., description="Estimated time to complete the task in hours")
    required_resources: List[str] = Field(..., description="List of resources required to complete the task")

class Milestone(BaseModel):
    milestone_name: str = Field(..., description="Name of the milestone")
    tasks: List[str] = Field(..., description="List of task IDs associated with this milestone")

class ProjectPlan(BaseModel):
    tasks: List[TaskEstimate] = Field(..., description="List of tasks with their estimates")
    milestones: List[Milestone] = Field(..., description="List of project milestones")

    

### Creating Agents and Tasks

In [25]:
agents_config['project_planning_agent']

{'role': 'The Ultimate Project Planner\n',
 'goal': 'To meticulously break down the {project_type} project into actionable tasks, ensuring no detail is overlooked, and setting precise timelines that align with the {project_objectives}.\n',
 'backstory': "As a veteran project manager, you’ve led numerous successful projects, particularly in {industry}. Your keen eye for detail and strategic thinking have always ensured that projects are delivered on time and within scope. Now, you're tasked with planning the next groundbreaking {project_type} project.\n",
 'allow_delegation': False,
 'verbose': True}

In [26]:
# Creating Agents
project_planning_agent = Agent(
  config=agents_config['project_planning_agent'],
  llm =  ollama_llm,
)

estimation_agent = Agent(
  config=agents_config['estimation_agent'],
  llm =  ollama_llm,
)

resource_allocation_agent = Agent(
  config=agents_config['resource_allocation_agent'],
  llm =  ollama_llm,
)

In [27]:
# Creating Tasks
task_breakdown = Task(
  config=tasks_config['task_breakdown'],
  agent=project_planning_agent
)

time_resource_estimation = Task(
  config=tasks_config['time_resource_estimation'],
  agent=estimation_agent
)

resource_allocation = Task(
  config=tasks_config['resource_allocation'],
  agent=resource_allocation_agent,
  output_pydantic=ProjectPlan # This is the structured output we want
)

In [28]:
# Creating Crew
crew = Crew(
  agents=[
    project_planning_agent,
    estimation_agent,
    resource_allocation_agent
  ],
  tasks=[
    task_breakdown,
    time_resource_estimation,
    resource_allocation
  ],
  verbose=True
)

### Crew's Input

In [29]:
from IPython.display import display, Markdown

project = 'Website'
industry = 'Technology'
project_objectives = 'Create a website for a small business'
team_members = """
- John Doe (Project Manager)
- John Smith (Frontend Developer)
- Emily Davis (Backend Developer)
- Jane Doe (Fullstack Developer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
"""
project_requirements = """
- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust
"""

# Format the dictionary as Markdown for a better display in Jupyter Lab
formatted_output = f"""
**Project Type:** {project}

**Project Objectives:** {project_objectives}

**Industry:** {industry}

**Team Members:**
{team_members}
**Project Requirements:**
{project_requirements}
"""
# Display the formatted output as Markdown
display(Markdown(formatted_output))


**Project Type:** Website

**Project Objectives:** Create a website for a small business

**Industry:** Technology

**Team Members:**

- John Doe (Project Manager)
- John Smith (Frontend Developer)
- Emily Davis (Backend Developer)
- Jane Doe (Fullstack Developer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)

**Project Requirements:**

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust



### Kicking off the crew

In [31]:
# The given Python dictionary
inputs = {
  'project_type': project,
  'project_objectives': project_objectives,
  'industry': industry,
  'team_members': team_members,
  'project_requirements': project_requirements
}

# Run the crew
result = crew.kickoff(
  inputs=inputs
)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 49416fa8-298e-4920-91b8-428b59186096                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 733748a7-c726-46d1-af51-782258d8cd0c
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 733748a7-c726-46d1-af51-782258d8cd0c
       Status: Executing Task...
    └── 🤖 Agent: The Ultimate Project Planner
        
            Status: In Progress

# Agent: The Ultimate Project Planner
## Task: Carefully analyze the project_requirements for the Website project and break them down into individual tasks. Define each task's scope in detail, set achievable timelines, and ensure that all dependencies are accounted for:

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build tru

🚀 Crew: crew
└── 📋 Task: 733748a7-c726-46d1-af51-782258d8cd0c
       Status: Executing Task...
    └── 🤖 Agent: The Ultimate Project Planner
        
            Status: ✅ Completed

...

# Agent: Resource Allocation Strategist
## Task: Strategically allocate tasks for the Website project to team members based on their skills, availability, and current workload. Ensure that each task is assigned to the most suitable team member and that the workload is evenly distributed.

Team members:

- John Doe (Project Manager)
- John Smith (Frontend Developer)
- Emily Davis (Backend Developer)
- Jane Doe (Fullstack Developer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)




# Agent: Resource Allocation Strategist
## Final Answer: 
{
  "tasks": [
    {
      "task_name": "Conduct competitor analysis and create design brief",
      "estimated_time_hours": 24,
      "required_resources": ["Bob Smith"]
    },
    {
      "task_name": "Develop wireframes and high-fidelity prototypes",
      "estimated_time_hours": 48,
      "required_resources": ["Bob Smith"]
    },
    {
      "task_name": "Implement responsive design using CSS Grid or Flexbox",
      "estimated_time_hours": 4

🚀 Crew: crew
├── 📋 Task: 733748a7-c726-46d1-af51-782258d8cd0c
│      Assigned to: The Ultimate Project Planner
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: The Ultimate Project Planner
│       
│           Status: ✅ Completed
├── 📋 Task: 642d9341-95d6-4024-b858-6b9c3d694c6d
│      Assigned to: Expert Estimation Analyst
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Expert Estimation Analyst
│       
│           Status: ✅ Completed
└── 📋 Task: d0378070-ff41-49a5-88d8-52fb5c0c1d13
       Status: Executing Task...
    └── 🤖 Agent: Resource Allocation Strategist
        
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 733748a7-c726-46d1-af51-782258d8cd0c
│      Assigned to: The Ultimate Project Planner
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: The Ultimate Project Planner
│       
│           Status: ✅ Completed
├── 📋 Task: 642d9341-95d6-4024-b858-6b9c3d694c6d
│      Assigned to: Expert Estimation Analyst
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Expert Estimation Analyst
│       
│           Status: ✅ Completed
└── 📋 Task: d0378070-ff41-49a5-88d8-52fb5c0c1d13
       Assigned to: Resource Allocation Strategist
    
       Status: ✅ Completed
    └── 🤖 Agent: Resource Allocation Strategist
        
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: d0378070-ff41-49a5-88d8-52fb5c0c1d13                                                                     │
│  Agent: Resource Allocation Strategist                                                                          │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 49416fa8-298e-4920-91b8-428b59186096                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

### Usage Metrics and Costs

In [34]:
crew.usage_metrics.prompt_tokens

13719

In [33]:
import pandas as pd

costs = 0.150 * (crew.usage_metrics.prompt_tokens + crew.usage_metrics.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([crew.usage_metrics.dict()])
df_usage_metrics

Total costs: $0.0034


/var/folders/xd/3z5vvpds0zxf_pypxd4cn3d80000gn/T/ipykernel_87480/1074044196.py:7: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  df_usage_metrics = pd.DataFrame([crew.usage_metrics.dict()])


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,22730,13719,0,9011,6


### Result

In [35]:
print(result.raw)

{
  "tasks": [
    {
      "task_name": "Conduct competitor analysis and create design brief",
      "estimated_time_hours": 24,
      "required_resources": ["Bob Smith"]
    },
    {
      "task_name": "Develop wireframes and high-fidelity prototypes",
      "estimated_time_hours": 48,
      "required_resources": ["Bob Smith"]
    },
    {
      "task_name": "Implement responsive design using CSS Grid or Flexbox",
      "estimated_time_hours": 40,
      "required_resources": ["John Smith"]
    },
    {
      "task_name": "Implement modern UI/UX using React or Angular",
      "estimated_time_hours": 32,
      "required_resources": ["John Smith"]
    },
    {
      "task_name": "Develop navigation system and menu structure",
      "estimated_time_hours": 24,
      "required_resources": ["John Smith"]
    },
    {
      "task_name": "Develop backend API using Node.js or Python",
      "estimated_time_hours": 40,
      "required_resources": ["Emily Davis"]
    },
    {
      "task_name": 

In [36]:
tasks = result.pydantic.dict()['tasks']
df_tasks = pd.DataFrame(tasks)

# Display the DataFrame as an HTML table
df_tasks.style.set_table_attributes('border="1"').set_caption("Task Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)

/var/folders/xd/3z5vvpds0zxf_pypxd4cn3d80000gn/T/ipykernel_87480/793840085.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  tasks = result.pydantic.dict()['tasks']


,task_name,estimated_time_hours,required_resources
0,Conduct competitor analysis and create design brief,24.000000,['Bob Smith']
1,Develop wireframes and high-fidelity prototypes,48.000000,['Bob Smith']
2,Implement responsive design using CSS Grid or Flexbox,40.000000,['John Smith']
3,Implement modern UI/UX using React or Angular,32.000000,['John Smith']
4,Develop navigation system and menu structure,24.000000,['John Smith']
5,Develop backend API using Node.js or Python,40.000000,['Emily Davis']
6,Implement authentication and authorization using Passport.js or Auth0,24.000000,['Emily Davis']
7,Conduct unit testing using Jest or Mocha,16.000000,['Alice Johnson']
8,Perform integration testing using Cypress or Selenium,24.000000,['Alice Johnson']
9,"Deploy website on hosting platform (AWS, DigitalOcean, etc.)",16.000000,['John Smith or Emily Davis']


### Inspecting Milestones

In [37]:
milestones = result.pydantic.dict()['milestones']
df_milestones = pd.DataFrame(milestones)

# Display the DataFrame as an HTML table
df_milestones.style.set_table_attributes('border="1"').set_caption("Task Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)

/var/folders/xd/3z5vvpds0zxf_pypxd4cn3d80000gn/T/ipykernel_87480/795124587.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  milestones = result.pydantic.dict()['milestones']


,milestone_name,tasks
0,Milestone 1,"['Conduct competitor analysis and create design brief', 'Develop wireframes and high-fidelity prototypes']"
1,Milestone 2,"['Implement responsive design using CSS Grid or Flexbox', 'Implement modern UI/UX using React or Angular']"
2,Milestone 3,"['Develop navigation system and menu structure', 'Develop backend API using Node.js or Python']"
3,Milestone 4,"['Implement authentication and authorization using Passport.js or Auth0', 'Conduct unit testing using Jest or Mocha']"
